### 1. Business Understanding

The dataset we will analyze here, consists of information about domestic flights operated by large airlines in the U.S. throughout 2008. It provides details about the schedule and delays of each flight. The data was originally  compiled and made available to the public by the Bureau of Transportation Statistics, but the version we will use here was obtained from Kaggle, at: 

[https://www.kaggle.com/giovamata/airlinedelaycauses](https://www.kaggle.com/giovamata/airlinedelaycauses)

Each record provides details about a specific flight, such as its number, origin, destination, airline, scheduled timeline and actual timeline. Information about delays (deviations from the scheduled timeline) is particularly intersting, as it provides us with the opportunity to analyze how delays are related to other factors and how much are domestic flights affected. The results will hopefully provide insights useful to passengers.

In view of the information at hand, we could consider digging for answers to the following questions:

- What are the major causes of delays and cancellations affecting domestic flights in the U.S.?


- Which are the best and the worst carriers (airlines) regarding delays and cancellations? Which are the carriers with the less/most cancellations and the shortest/longest delays?


- Can we identify periods of the year 2008 when delays were particularly long? For instance, days of the week or months of the year.


- Are there any combinations of variables that appear to determine the expected delay time or probability of cancellation of a flight?


- Could we estimate the probability of a flight being cancelled based on some of the variables?

The answers we arrive at will hopefully help passengers make more informed decisions in their travels and avoid the inconveniences of getting into a delayed flight. 

### 2. Data Understanding

By invoking the info function on the DataFrame, we see that it contains 1936758 records and 30 variables:

In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

df_flight_delays = pd.read_csv("datasets/DelayedFlights.csv")

print(f"Airline Delays. Records: {df_flight_delays.shape[0]}, variables: {df_flight_delays.shape[1]}")

# Get the size of the dataset, list of the variables with their types
df_flight_delays.info()

Airline Delays. Records: 1936758, variables: 30
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1936758 entries, 0 to 1936757
Data columns (total 30 columns):
Unnamed: 0           int64
Year                 int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
DepTime              float64
CRSDepTime           int64
ArrTime              float64
CRSArrTime           int64
UniqueCarrier        object
FlightNum            int64
TailNum              object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             int64
TaxiIn               float64
TaxiOut              float64
Cancelled            int64
CancellationCode     object
Diverted             int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    floa

The following table provides a description and the type of each variable:

Number | Name | Description | Type
:------: | :---- | :---- | :----
1 | (Unamed) | Sequential record number | Numerical, integer
2 | Year | 2008 | Numerical, integer
3 | Month | Month number (1=January - 12=December) | Numerical, integer
4 | DayofMonth | Day number of the month (1 - 31) | Numerical, integer
5 | DayOfWeek | Day number of the week (1=Monday - 7=Sunday) | Numerical, integer
6 | DepTime | Actual departure time (local, hhmm) | Numerical, integer
7 | CRSDepTime | Scheduled departure time (local, hhmm) | Numerical, integer
8 | ArrTime | Actual arrival time (local, hhmm) | Numerical, integer
9 | CRSArrTime | Scheduled arrival time (local, hhmm) | Numerical, integer
10 | UniqueCarrier | Unique carrier code | Categorical, string
11 | FlightNum | Flight number | Numerical, integer
12 | TailNum | Plane tail number | Categorical, string
13 | ActualElapsedTime | Actual total flight time, in minutes | Numerical, decimal
14 | CRSElapsedTime | Total flight time estimated by the carrier, in minutes | Numerical, decimal
15 | AirTime | Time in the air, in minutes | Numerical, decimal
16 | ArrDelay | Delay to arrive at the destination airport, in minutes | Numerical, decimal
17 | DepDelay | Delay to depart from the origin airport, in minutes | Numerical, decimal
18 | Origin | Origin IATA airport code | Categorical, string
19 | Dest | Destination IATA airport code | Categorical, string
20 | Distance | Travel in miles | Numerical, integer
21 | TaxiIn | Taxi in time (time to roll from the terminal to the runway), in minutes | Numerical, decimal
22 | TaxiOut | Taxi out time (time to roll from the runway to the terminal) in minutes | Numerical, decimal
23 | Cancelled | Was the flight cancelled? | Numerical, boolean
24 | CancellationCode | Reason why flight was cancelled (A=carrier, B=weather, C=NAS, D=security) | Categorical, string
25 | Diverted | Was the flight diverted to another destination? (1=yes, 0=no) | Numerical, integer
26 | CarrierDelay | Delay caused by the carrier, in minutes | Numerical, decimal
27 | WeatherDelay | Delay caused by the weather, in minutes | Numerical, decimal
28 | NASDelay | Delay caused by the NAS (National Airspace System), in minutes | Numerical, decimal
29 | SecurityDelay | Delay caused by security issues, in minutes | Numerical, decimal
30 | LateAircraftDelay | Delay caused by the aircraft arriving late, in minutes | Numerical, decimal

There are five categorical variables: UniqueCarrier, TailNum, Origin, Dest and CancellationCode. The rest are all numerical. However, when performing operations on DepTime, CRSDepTime, ArrTime and CRSArrTime we must bear in mind that these represent times in hour-minute format.

In [63]:
# Get the names of all categorical variables
print(f"Categorical variables: {df_flight_delays.select_dtypes(include=['object']).columns.values}")

Categorical variables: ['UniqueCarrier' 'TailNum' 'Origin' 'Dest' 'CancellationCode']


### 3. Data Preparation

Now we will examine the data in search for errors and inconsistencies and modify it as necessary to facilitate analysis.

Note that the first variable (Unamed: 0) just enumerates the rows. It can be safely discarded as it provides no information useful for our analysis. We can also drop the Year variable, as all observations are of the same year (2008) in this version of the dataset.

In [61]:
# Drop the first column, which just provides enumerates the rows. The DataFrame index does so already
df_flight_delays.drop(["Unnamed: 0", "Year"], axis=1, inplace=True)

Now let's check for missing values. We start by counting how many we have in each variable:

In [62]:
# Count the total missing entries per column
df_nan_counts = df_flight_delays.isna().sum()

# Compute the percentage of missing values per column
df_nan_percents = (df_nan_counts / df_flight_delays.shape[0]) * 100

df_nan_data = pd.DataFrame({"CountMissing": df_nan_counts, "PercentMissing": df_nan_percents}, 
                           index=df_nan_counts.index)

print(df_nan_data)

                   CountMissing  PercentMissing
Month                         0        0.000000
DayofMonth                    0        0.000000
DayOfWeek                     0        0.000000
DepTime                       0        0.000000
CRSDepTime                    0        0.000000
ArrTime                    7110        0.367108
CRSArrTime                    0        0.000000
UniqueCarrier                 0        0.000000
FlightNum                     0        0.000000
TailNum                       5        0.000258
ActualElapsedTime          8387        0.433043
CRSElapsedTime              198        0.010223
AirTime                    8387        0.433043
ArrDelay                   8387        0.433043
DepDelay                      0        0.000000
Origin                        0        0.000000
Dest                          0        0.000000
Distance                      0        0.000000
TaxiIn                     7110        0.367108
TaxiOut                     455        0

Since several of the questions we aim at answering are concerned with the schedule of flights, the variables ArrTime, ActualElapsedTime, CRSElapsedTime, AirTime, ArrDelay, TaxiIn, TaxiOut, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay and LateAircraftDelay are particularly interesting to us. Hence, we should examine why do they have missing values and handle them properly.

**Flights with missing ArrTime**

Taking a look at some of the flights that have missing ArrTime values, it appears that all of them are either cancelled or diverted.

In [68]:
# Get flights that have ArrTime missing and print the first 5
df_missing_arrtime = df_flight_delays[df_flight_delays["ArrTime"].isna()]
df_missing_arrtime[["Cancelled", "Diverted"]].head()

,Cancelled,Diverted
1280,0,1
1372,0,1
1776,0,1
1831,0,1
2244,0,1


We can verify if that's really the case by counting how many of such flights are cancelled or diverted.

In [74]:
# Count how many of the flights missing ArrTime were also cancelled or diverted
total_missing_arrtime_diverted_or_cancelled = sum((df_missing_arrtime["Cancelled"] == 1.0) | 
                                                  (df_missing_arrtime["Diverted"] == 1.0))

print(f"Total missing ArrTime: {df_missing_arrtime.shape[0]}. Cancelled/Diverted: {total_missing_arrtime_diverted_or_cancelled}")

Total missing ArrTime: 7110. Cancelled/Diverted: 7110


This means that the dataset does not provide arrival times for flights that were cancelled or diverted. 

**Flights with missing ActualElapsedTime, AirTime and ArrDelay**

Interestingly, these three variables have the same number of missing values. This strongly suggests that if a flight is missing the value of any of these variables, it's actually missing the values of all three. We can easily verify this by comparing how many rows have all vs. any of these variables missing.

In [81]:
# Get rows with missing values in any of the variables ActualElapsedTime, AirTime or ArrDelay
df_missing_any_3 = df_flight_delays[(df_flight_delays["ActualElapsedTime"].isna()) |
                                  (df_flight_delays["AirTime"].isna()) |
                                  (df_flight_delays["ArrDelay"].isna()) | df_flight_delays["AirTime"].isna()]

# Get rows with missing values in all of the variables ActualElapsedTime, AirTime and ArrDelay
df_missing_all_3 = df_flight_delays[(df_flight_delays["ActualElapsedTime"].isna()) &
                                  (df_flight_delays["AirTime"].isna()) &
                                  (df_flight_delays["ArrDelay"].isna()) & df_flight_delays["AirTime"].isna()]

print(f"Total missing all ActualElapsedTime, AirTime and ArrDelay: {df_missing_all_3.shape[0]}. Total missing any: {df_missing_any_3.shape[0]}")

Total missing all ActualElapsedTime, AirTime and ArrDelay: 8387. Total missing any: 8387


Moreover, inspecting the rows we see that apparently all these flights were either cancelled or diverted too.

In [83]:
df_missing_any[["Cancelled", "Diverted"]].head()

,Cancelled,Diverted
1280,0,1
1372,0,1
1776,0,1
1831,0,1
2244,0,1


In [23]:
df_all_dly_details_na = df_flight_delays[df_flight_delays["CarrierDelay"].isna() &
                                         df_flight_delays["WeatherDelay"].isna() & df_flight_delays["NASDelay"].isna() &
                                         df_flight_delays["SecurityDelay"].isna() &
                                         df_flight_delays["LateAircraftDelay"].isna()]

df_all_dly_details_na.head()

,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,...,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,1,2008,1,3,4,754.0,735,1002.0,1000,WN,...,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2,2008,1,3,4,628.0,620,804.0,750,WN,...,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
4,5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,...,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
6,10,2008,1,3,4,706.0,700,916.0,915,WN,...,5.0,19.0,0,N,0,NaN,NaN,NaN,NaN,NaN
